In [ ]:
#source activate newEnv
install.packages("broom")
if (!requireNamespace('BiocManager', quietly = TRUE))
    install.packages('BiocManager')
BiocManager::install('parallelly')
install.packages("Seurat")

library(dplyr)
library(ggplot2)
library(sctransform)
library(scater)
library(reticulate)
library(future)
#library('gPCA')
library('Biobase')
library(pheatmap)
#library("ggfortify")
#library('qvalue')
library(gplots)
#library('DESeq2')
#library(VennDiagram)
library('hdf5r')
library(EnsDb.Hsapiens.v86)

library(BiocParallel)
library(tictoc)
library(Seurat)
library(Signac)
library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(cowplot)
library(GenomeInfoDb)

suppressMessages(library(hdf5r))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(EnsDb.Hsapiens.v86))
suppressMessages(library(dplyr))
suppressMessages(library(ggplot2))
suppressMessages(library(Matrix))
suppressMessages(library(harmony))
suppressMessages(library(data.table))
suppressMessages(library(ggpubr))
suppressMessages(library(future))
library(tidyr)

In [ ]:
install.packages("ggpubr")
install.packages("ggbreak")
install.packages("gridExtra")
install.packages("grid")
install.packages("ggh4x")
library(ggpubr)
library(ggbreak)
library(gridExtra)
library(grid)
library(ggh4x)
library(ggplot2)
library(ggforce)

### Read in the  peaks  lfm


In [ ]:
samples <- c('HPAP-035', 'HPAP-036', 'HPAP-039', 'HPAP-040', 'HPAP-044', 'HPAP-045', 'HPAP-047', 'HPAP-049', 'HPAP-050', 'HPAP-051', 'HPAP-052', 'HPAP-053', 'HPAP-054', 'HPAP-055', 'HPAP-056', 'HPAP-059', 'HPAP-061', 'HPAP-062', 'HPAP-063', 'HPAP-064', 'HPAP-067', 'HPAP-069', 'HPAP-072', 'HPAP-075', 'HPAP-077', 'HPAP-079', 'HPAP-080', 'HPAP-081', 'HPAP-083', 'HPAP-084', 'HPAP-085', 'HPAP-088', 'HPAP-092', 'HPAP-099', 'HPAP-100', 'HPAP-101', 'HPAP-103', 'HPAP-104', 'HPAP-105', 'HPAP-106', 'HPAP-109')


### Read in long fmt mtx V1 should be windows, V2 should be barcodes

In [ ]:
.libPaths()

In [ ]:
BiocManager::install("Bioconductor/GenomeInfoDb",lib = "/home/parulk/R/x86_64-pc-linux-gnu-library/4.1",force = TRUE)

In [ ]:
library(GenomeInfoDb,lib.loc="/home/parulk/R/x86_64-pc-linux-gnu-library/4.1")

In [ ]:
packageVersion("GenomeInfoDb",lib.loc="/home/parulk/R/x86_64-pc-linux-gnu-library/4.1")

In [ ]:
GenomeInfoDb::getChromInfoFromUCSC("hg38")

In [ ]:
#### Read in the ATAC data to Seurat using the new lfm matrices
#read in ATAC data from the lfm matrices (sm workaround method for now)
# load in starting ATAC long format matrices to a list 
atacs_OG <- list()
atacs_FINAL <- list()

for (sample in samples) {
    #print(sample)
    wd <- sprintf('/nfs/lab/parulk/HPAP_scATAC/peak_calls_leiden/lfm/', sample)
    atacs_OG[[sample]] <- read.table(file.path(wd, sprintf('%s.long_fmt_mtx.txt.gz',sample)), sep='\t', header=FALSE, stringsAsFactors=FALSE)
    #atacs_OG[[sample]]$V1 <- as.factor(atacs_OG[[sample]]$V1)
    #atacs_OG[[sample]]$V2 <- as.factor(atacs_OG[[sample]]$V2)
    atacs_FINAL[[sample]] <- read.table(file.path(wd, sprintf('%s.long_fmt_mtx.txt.gz',sample)), sep='\t', header=FALSE, stringsAsFactors=FALSE)
    atacs_FINAL[[sample]]$V1 <- as.factor(atacs_OG[[sample]]$V2)
    atacs_FINAL[[sample]]$V2 <- as.factor(atacs_OG[[sample]]$V1)
    atacs_OG[[sample]] <- NULL
    #atacs_FINAL[[sample]] <- atacs_FINAL[[sample]][atacs_FINAL[[sample]]$V2 %in% good,]
}

In [ ]:
atac_mod <- atacs_FINAL
head(atac_mod)

In [ ]:
#### ADD '-1' TO THE BARCODES FROM JOSHS MATRICES 
# THIS IS ALL BECAUSE THE FRAGMENT FILE FROM CELLRANGER IS IN THIS FORMAT AND SEURAT NEEDS THEM TO MATCH
atac_mod$`HPAP-035`$V2 <- paste0(atac_mod$`HPAP-035`$V2, "-1")
atac_mod$`HPAP-036`$V2 <- paste0(atac_mod$`HPAP-036`$V2, "-1")
atac_mod$`HPAP-039`$V2 <- paste0(atac_mod$`HPAP-039`$V2, "-1")
atac_mod$`HPAP-040`$V2 <- paste0(atac_mod$`HPAP-040`$V2, "-1")
atac_mod$`HPAP-044`$V2 <- paste0(atac_mod$`HPAP-044`$V2, "-1")
atac_mod$`HPAP-045`$V2 <- paste0(atac_mod$`HPAP-045`$V2, "-1")
atac_mod$`HPAP-047`$V2 <- paste0(atac_mod$`HPAP-047`$V2, "-1")
atac_mod$`HPAP-049`$V2 <- paste0(atac_mod$`HPAP-049`$V2, "-1")
atac_mod$`HPAP-050`$V2 <- paste0(atac_mod$`HPAP-050`$V2, "-1")
atac_mod$`HPAP-051`$V2 <- paste0(atac_mod$`HPAP-051`$V2, "-1")
atac_mod$`HPAP-052`$V2 <- paste0(atac_mod$`HPAP-052`$V2, "-1")
atac_mod$`HPAP-053`$V2 <- paste0(atac_mod$`HPAP-053`$V2, "-1")
atac_mod$`HPAP-054`$V2 <- paste0(atac_mod$`HPAP-054`$V2, "-1")
atac_mod$`HPAP-055`$V2 <- paste0(atac_mod$`HPAP-055`$V2, "-1")
atac_mod$`HPAP-056`$V2 <- paste0(atac_mod$`HPAP-056`$V2, "-1")
atac_mod$`HPAP-059`$V2 <- paste0(atac_mod$`HPAP-059`$V2, "-1")
atac_mod$`HPAP-061`$V2 <- paste0(atac_mod$`HPAP-061`$V2, "-1")
atac_mod$`HPAP-062`$V2 <- paste0(atac_mod$`HPAP-062`$V2, "-1")
atac_mod$`HPAP-063`$V2 <- paste0(atac_mod$`HPAP-063`$V2, "-1")
atac_mod$`HPAP-064`$V2 <- paste0(atac_mod$`HPAP-064`$V2, "-1")
atac_mod$`HPAP-067`$V2 <- paste0(atac_mod$`HPAP-067`$V2, "-1")
atac_mod$`HPAP-069`$V2 <- paste0(atac_mod$`HPAP-069`$V2, "-1")
atac_mod$`HPAP-072`$V2 <- paste0(atac_mod$`HPAP-072`$V2, "-1")
atac_mod$`HPAP-075`$V2 <- paste0(atac_mod$`HPAP-075`$V2, "-1")
atac_mod$`HPAP-077`$V2 <- paste0(atac_mod$`HPAP-077`$V2, "-1")
atac_mod$`HPAP-079`$V2 <- paste0(atac_mod$`HPAP-079`$V2, "-1")
atac_mod$`HPAP-080`$V2 <- paste0(atac_mod$`HPAP-080`$V2, "-1")
atac_mod$`HPAP-081`$V2 <- paste0(atac_mod$`HPAP-081`$V2, "-1")
atac_mod$`HPAP-083`$V2 <- paste0(atac_mod$`HPAP-083`$V2, "-1")
atac_mod$`HPAP-084`$V2 <- paste0(atac_mod$`HPAP-084`$V2, "-1")
atac_mod$`HPAP-085`$V2 <- paste0(atac_mod$`HPAP-085`$V2, "-1")
atac_mod$`HPAP-088`$V2 <- paste0(atac_mod$`HPAP-088`$V2, "-1")
atac_mod$`HPAP-092`$V2 <- paste0(atac_mod$`HPAP-092`$V2, "-1")
atac_mod$`HPAP-099`$V2 <- paste0(atac_mod$`HPAP-099`$V2, "-1")
atac_mod$`HPAP-100`$V2 <- paste0(atac_mod$`HPAP-100`$V2, "-1")
atac_mod$`HPAP-101`$V2 <- paste0(atac_mod$`HPAP-101`$V2, "-1")
atac_mod$`HPAP-103`$V2 <- paste0(atac_mod$`HPAP-103`$V2, "-1")
atac_mod$`HPAP-104`$V2 <- paste0(atac_mod$`HPAP-104`$V2, "-1")
atac_mod$`HPAP-105`$V2 <- paste0(atac_mod$`HPAP-105`$V2, "-1")
atac_mod$`HPAP-106`$V2 <- paste0(atac_mod$`HPAP-106`$V2, "-1")
atac_mod$`HPAP-109`$V2 <- paste0(atac_mod$`HPAP-109`$V2, "-1")

### Read in peaks file

In [ ]:
#read in the HVWs set, we'll cut down all samples to be these windows
hvw_fp2 <- '/nfs/lab/parulk/HPAP_scATAC/peak_calls_leiden/peaks/mergedPeak_formatted.txt'
hvws2 <- scan(hvw_fp2, what="", sep="\n")
print(head(hvws2))

#sort alphanumerically
hvws_fin2 <- sort(hvws2)
print(tail(hvws_fin2))

In [ ]:
#function which takes in a list of long format atac_fragment dfs with 
#sample names (df), an overall windows file (windows) or peaks file and then makes these 
#into sparse matrices and merges them together

#modified to take in the hvws set and use those... will still check if 
#there's any missing windows and add those, so only a few changes!

merge_sparse_matrices_hvws <- function(dfs, windows){
    samples <- names(dfs)
    for (sample in samples){
        #get missing windows list for this sample
        print(paste(sample,Sys.time(),sep=': '))
        df <- dfs[[sample]]
        mis_windows <- windows[!windows %in% levels(df$V1)]
        
        #make sure there are missing_windows
        if (length(mis_windows) > 0){
            print('Adding missing windows')
            #create a new long format matrix (sm) with the missing windows added as 0 counts
            filler_bc <- as.character(df$V2[[1]])
            print(paste("Using filler BC:",filler_bc,sep=" "))
            new_rows <- cbind(as.data.frame(mis_windows),
                              as.data.frame(rep(filler_bc),length(mis_windows)),
                              as.data.frame(rep(0,length(mis_windows))))
            colnames(new_rows) <- c("V1","V2","V3")
            lfm <- rbind(df,new_rows)
        #if there aren't, set lfm to df
        } else {
            print('No windows were missing')
            lfm <- df
        }
        
        #cut down lfm to just be the hvws (windows)
        lfm_cut <- lfm[lfm$V1 %in% windows,]
        
        #cut down barcodes to keep those in final set 
        #lfm_cut <- lfm_cut[lfm_cut$V2 %in% good,]
        #lfm_cut <- lfm[,lfm$V2 %in% good]
        #atacs_FINAL[[sample]] <- atacs_FINAL[[sample]][atacs_FINAL[[sample]]$V2 %in% good,]
        #set the levels of the lfm based on the desired bc order and reorder V1
        lfm_cut$V1 <- factor(lfm_cut$V1, levels=windows)
        lfm2 <- lfm_cut[order(lfm_cut$V1),]
        lfm2$V2 <- as.factor(lfm2$V2)
        
        if (sample == samples[1]){
            #if first sample, will make the overall sparse matrix 
            overall_sm <- with(lfm2,sparseMatrix(i=as.numeric(V1), j=as.numeric(V2), x=V3, dimnames=list(levels(V1), levels(V2))))
            print(dim(overall_sm))
            
        } else {
            #lfm2 <- lfm2[lfm2$V2 %in% good,]

            #otherwise, convert into a sparse matrix and add to the overall one
            sm = with(lfm2,sparseMatrix(i=as.numeric(V1), j=as.numeric(V2), x=V3, dimnames=list(levels(V1), levels(V2))))
            print(dim(sm))
            overall_sm = cbind(overall_sm, sm) 
        }
    }
    return(overall_sm)
}

In [ ]:
overall_sm2 <- merge_sparse_matrices_hvws(atac_mod,hvws_fin2)
dim(overall_sm2)
head(overall_sm2)

In [ ]:
#check if the windows of overall_sm are sorted (they should be)
sorted_windows2 <- sort(row.names(overall_sm2))
table(sorted_windows2 == row.names(overall_sm2))

In [ ]:
## atac == giant matrix of barcodes by highly varibale windows
atac <- overall_sm2


In [ ]:
#continue to make Seurat compatible object
suppressMessages(annotations <- GetGRangesFromEnsDb(ensdb=EnsDb.Hsapiens.v86))
seqlevelsStyle(annotations) <- 'UCSC'
genome(annotations) <- 'hg38'


In [ ]:
frag.file ='/nfs/lab/parulk/HPAP_scATAC/merged_samples-new.bed.gz'
head(frag.file)

In [ ]:
atac_assay <- CreateChromatinAssay(counts=atac, sep=c(':', '-'), genome='hg38', fragments=frag.file, min.cells=0, min.features=-1, annotation=annotations)
head(atac_assay)

In [ ]:
Atac <-CreateSeuratObject(atac_assay, project = "HPAP", assay = "ATAC_peaks",
  min.cells = 0, min.features = 0, names.field = 2,
  names.delim = "_")
head(Atac)
str(Atac)

In [ ]:
### Since our sampleIDs are structured HPAP- we cut off the barcode bit of each row and store as the library
Atac$library <- gsub('.{19}$', '', rownames(Atac@meta.data))
head(Atac$library)

In [ ]:
### add whatever metadata for harmony
### add whatever metadata for harmony
sex_F = list('HPAP-036', 'HPAP-039', 'HPAP-044','HPAP-045', 'HPAP-050', 'HPAP-051', 'HPAP-053', 'HPAP-054', 'HPAP-062', 'HPAP-063', 'HPAP-069', 'HPAP-079', 'HPAP-081', 'HPAP-084', 'HPAP-085', 'HPAP-099', 'HPAP-101', 'HPAP-103', 'HPAP-105', 'HPAP-109')
cond_t1d = list('HPAP-055', 'HPAP-064', 'HPAP-084')
cond_t2d = list('HPAP-051', 'HPAP-061', 'HPAP-062', 'HPAP-079', 'HPAP-081', 'HPAP-083', 'HPAP-085', 'HPAP-088', 'HPAP-100', 'HPAP-106', 'HPAP-109')
cond_ctrl = list('HPAP-035', 'HPAP-036', 'HPAP-039', 'HPAP-040', 'HPAP-044', 'HPAP-045', 'HPAP-047', 'HPAP-049', 'HPAP-050', 'HPAP-052', 'HPAP-053', 'HPAP-054', 'HPAP-056', 'HPAP-059', 'HPAP-063', 'HPAP-067', 'HPAP-069', 'HPAP-072', 'HPAP-075', 'HPAP-077', 'HPAP-080', 'HPAP-092', 'HPAP-099', 'HPAP-101', 'HPAP-103', 'HPAP-104', 'HPAP-105')           




Atac@meta.data$sex[Atac@meta.data$library %in% sex_F] <- 'F'
Atac@meta.data$sex[!Atac@meta.data$library %in% sex_F] <- 'M'

Atac@meta.data$condition[Atac@meta.data$library %in% cond_t1d] <- 'T1D'
Atac@meta.data$condition[Atac@meta.data$library %in% cond_t2d] <- 'T2D'
Atac@meta.data$condition[Atac@meta.data$library %in% cond_ctrl] <- 'Control'
head(Atac@meta.data)
head(Atac@meta.data$condition)
head(Atac@meta.data$sex)

In [ ]:
#### I save everything up to here because Seurat likes to crash and its annoying to re do everything

saveRDS(Atac, file='/nfs/lab/parulk/HPAP_scATAC/peak_based_intermediate.rds')



In [ ]:
Atac <- readRDS('/nfs/lab/parulk/HPAP_scATAC/peak_based_intermediate.rds')

In [ ]:
Atac

In [ ]:
head(Atac)

In [ ]:
DefaultAssay(Atac) <- 'ATAC_peaks'
Atac <- RunTFIDF(Atac)
Atac <- FindTopFeatures(Atac, min.cutoff='q0', verbose=FALSE)





In [ ]:
Atac <- RunSVD(Atac)

In [ ]:
DepthCor(Atac)

In [ ]:
hm_atac <- HarmonyMatrix(Embeddings(Atac, reduction='lsi'),Atac@meta.data,  c("library","sex"), do_pca=FALSE,plot_convergence = TRUE, verbose = TRUE)


In [ ]:
Atac[['harmony.atac']] <- CreateDimReducObject(embeddings=hm_atac, key='LSI_', assay= 'ATAC_peaks')

In [ ]:
Atac <- RunUMAP(Atac, dims=2:30, reduction='harmony.atac', reduction.name='umap.atac', reduction.key='atacUMAP_')


In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
p3 <- DimPlot(Atac, reduction='umap.atac', group.by = 'library', label=TRUE, label.size=6, repel=TRUE, raster=FALSE) + ggtitle('WNN')
p3 <- p3 + xlab('UMAP 1') + ylab('UMAP 2') + ggtitle('ATAC')
p3

In [ ]:
Atac@reductions

In [ ]:
DepthCor(Atac, reduction = 'harmony.atac')

In [ ]:
Atac <- FindNeighbors(object = Atac, reduction = 'harmony.atac', dims = 2:30)

In [ ]:
Atac <- FindClusters(object = Atac, algorithm=4,resolution = 1.5,method = "igraph") 

In [ ]:
DimPlot(object = Atac, label = TRUE) + NoLegend()


In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
#DimPlot(object = Atac, group.by = 'technology',label = FALSE)# + NoLegend()
DimPlot(object = Atac, group.by = 'library',label = FALSE)# + NoLegend()
DimPlot(object = Atac, group.by = 'condition',label = FALSE)# + NoLegend()



In [ ]:
saveRDS(Atac, file='/nfs/lab/parulk/HPAP_scATAC/peaks_all_samples_harmony_reduced_final.rds')


In [ ]:
#Read object after processing
atac_obj <- readRDS('/nfs/lab/parulk/HPAP_scATAC/peaks_all_samples_harmony_reduced_final.rds')

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
DimPlot(atac_obj, label = TRUE, repel=TRUE)

In [ ]:
atac_obj

In [ ]:
#gene activity
gene.activities <- GeneActivity(atac_obj)
# add the gene activity matrix to the Seurat object as a new assay and normalize it
atac_obj[['RNA']] <- CreateAssayObject(counts = gene.activities)
atac_obj <- NormalizeData(
  object = atac_obj,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(atac_obj$nCount_RNA)
)

In [ ]:
atac_obj <- NucleosomeSignal(object = atac_obj)
atac_obj <- TSSEnrichment(object = atac_obj, fast = FALSE)
head(atac_obj@meta.data)

In [ ]:
#check
# add qc metrics from josh
samples <- c('HPAP-035', 'HPAP-036', 'HPAP-039', 'HPAP-040', 'HPAP-044', 'HPAP-045', 'HPAP-047', 'HPAP-049', 'HPAP-050', 'HPAP-051', 'HPAP-052', 'HPAP-053', 'HPAP-054', 'HPAP-055', 'HPAP-056', 'HPAP-059', 'HPAP-061', 'HPAP-062', 'HPAP-063', 'HPAP-064', 'HPAP-067', 'HPAP-069', 'HPAP-072', 'HPAP-075', 'HPAP-077', 'HPAP-079', 'HPAP-080', 'HPAP-081', 'HPAP-083', 'HPAP-084', 'HPAP-085', 'HPAP-088', 'HPAP-092', 'HPAP-099', 'HPAP-100', 'HPAP-101', 'HPAP-103', 'HPAP-104', 'HPAP-105', 'HPAP-106', 'HPAP-109')
qcs <- list()
for (sample in samples) {
    wd <- sprintf('/nfs/lab/parulk/HPAP_scATAC/lfm1/')
    qc <- read.table(file.path(wd, sprintf('%s.qc_metrics.txt', sample, sample)), sep='\t', header=TRUE, stringsAsFactors=FALSE)
    #qc <- qc[qc$is_cell==1,]
    #qc$X <- paste0(sample, '_', qc$barcodes)
    qcs[[sample]] <- qc
}
qc <- as.data.frame(rbindlist(qcs))
qc$X <- paste0(qc$X, "-1")

head(qc)
rownames(qc) <- qc$X
qc <- qc[Cells(atac_obj), 6:length(colnames(qc))]
atac_obj <- AddMetaData(object=atac_obj, metadata=qc)
qc <- qcs <- NULL
gc()
head(atac_obj)
metadata <- atac_obj@meta.data
head(atac_obj@meta.data)

In [ ]:
atac_obj

In [ ]:
metadata <- atac_obj@meta.data
frac_reads_in_peaks <- atac_obj@meta.data$frac_reads_in_peaks
quantile(frac_reads_in_peaks)
ggplot(data=metadata, mapping = aes(x=frac_reads_in_peaks)) +  geom_density(alpha = 0.2, color="green", fill="lightgreen")  + 
theme_linedraw()

In [ ]:
frac_reads_in_promoters <- atac_obj@meta.data$frac_reads_in_promoters
quantile(frac_reads_in_promoters)
ggplot(data=metadata, mapping = aes(x=frac_reads_in_promoters)) +  geom_density(alpha = 0.2, fill= 'lightpink', color="pink") +
theme_linedraw()

In [ ]:
TSS.enrichment <- atac_obj@meta.data$TSS.enrichment
quantile(TSS.enrichment)
ggplot(data=metadata, mapping = aes(x=TSS.enrichment)) +  geom_density(alpha = 0.2, color="blue", fill="lightblue") + 
theme_linedraw()
atac_obj$high.tss <- ifelse(atac_obj$TSS.enrichment > 2, 'High', 'Low')
TSSPlot(atac_obj, group.by = 'high.tss') + NoLegend()

In [ ]:
ggplot(data=metadata, mapping = aes(x=nucleosome_signal)) +  geom_density(alpha = 0.2,color="yellow", fill="yellow") + 
theme_linedraw()
atac_obj$nucleosome_group <- ifelse(atac_obj$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
FragmentHistogram(object = atac_obj, group.by = 'nucleosome_group')

In [ ]:
head(atac_obj)
ggplot(data=metadata, mapping = aes(x=frac_mito_reads)) +  geom_density(alpha = 0.2, color="orange", fill="orange") + 
theme_linedraw() 
frac_mito_reads <- atac_obj@meta.data$frac_mito_reads
quantile(frac_mito_reads)


In [ ]:
options(repr.plot.width=20, repr.plot.height=15)
p1 <- VlnPlot(atac_obj, features='TSS.enrichment',  pt.size=0, log = TRUE, split.by = 'seurat_clusters' ) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(atac_obj$TSS.enrichment), linetype='dashed')#
p2 <- VlnPlot(atac_obj, features='nucleosome_signal',  pt.size=0, log = TRUE, split.by = 'seurat_clusters' ) + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(atac_obj$nucleosome_signal), linetype='dashed')#
p3 <- VlnPlot(atac_obj, features='frac_reads_in_peaks',  pt.size=0,  split.by = 'seurat_clusters') + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(atac_obj$frac_reads_in_peaks), linetype='dashed')#
p4 <- VlnPlot(atac_obj, features='frac_reads_in_promoters',  pt.size=0, split.by = 'seurat_clusters') + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(atac_obj$frac_reads_in_promoters), linetype='dashed')#
p5 <- VlnPlot(atac_obj, features='frac_mito_reads',  pt.size=0, split.by = 'seurat_clusters') + geom_boxplot(width=.6, fill='white', alpha=.6) + geom_hline(yintercept=median(atac_obj$frac_mito_reads), linetype='dashed')#

p1/p2/p3/p4/p5

In [ ]:
atac_obj@reductions

In [ ]:
atac_obj

In [ ]:
saveRDS(atac_obj,'/nfs/lab/parulk/HPAP_scATAC/peak_based_metadata_geneactivity_TSS_added.rds')

In [ ]:
atac_obj <- readRDS('/nfs/lab/parulk/HPAP_scATAC/peak_based_metadata_geneactivity_TSS_added.rds')

In [ ]:
names(atac_obj@graphs)

In [ ]:
table(atac_obj[[]]$library)

In [ ]:
table(Idents(atac_obj))

In [ ]:
# Load markers list
project.dir = "/nfs/lab/parulk/HPAP_scATAC/"
cell.markers = read.table("/nfs/lab/parulk/HPAP_scATAC/Cell.markers.txt", sep = ',', header = TRUE)
# Make it long, remove useless column and void markers
cell.markers <- cell.markers %>% gather(Key, marker, c(3:ncol(cell.markers)))
cell.markers = cell.markers[,-3]
cell.markers = cell.markers[cell.markers$marker != "", ]
head(cell.markers)
# Factorize columns
cell.markersCompartment <- cell.markers$cell.markersCompartment
cell.markersCellType <- cell.markers$cell.markersCellType
cell.markersCompartment = factor(cell.markersCompartment, levels = c("Endocrine cells", "Non-endocrine cells"))
cell.markersCellType = factor(cell.markersCellType, levels = c("Beta", "Alpha", "Delta", "Gamma", "Epsilon", "Ductal", "MUC5B Ductal", "Acinar", "Stellate", "Act. Stellate", "Q. Stellate", "Endothelial", "T Cell", "Schwann", "Macrophages", "Dividing Cells"))

In [ ]:
g = DotPlot(atac_obj, assay='RNA', features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    options(repr.plot.width=25, repr.plot.height=10)
    figure <- ggplot(meta_summary, aes(x = marker, y = id)) +
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
      ylab("Cluster") + xlab("") +
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
facet_nested(~ Compartment + CellType, scales = "free")

figure

In [ ]:
options(repr.plot.width=8, repr.plot.height=10)
sample.abundance = as.data.frame(table(atac_obj$seurat_clusters, atac_obj$library))
colnames(sample.abundance) = c("cluster", "sample", "Freq")
ggplot(sample.abundance, aes(fill = sample, y = Freq, x = cluster)) +
  theme_bw() +
  coord_flip() +
  geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
  xlab('') + ylab('percentage')

In [ ]:
# Visualize integration
gg1 = DimPlot(atac_obj, group.by = "library", label = FALSE, pt.size = .7)
gg2 = DimPlot(atac_obj, group.by = "condition", label = FALSE, pt.size = .7)
gg3 = DimPlot(atac_obj, group.by = "sex", label = FALSE, pt.size = .7)

In [ ]:
options(repr.plot.height = 20, repr.plot.width = 40)
combined_plot <- gg1 + gg2 + gg3
combined_plot

In [ ]:
covariant.ls = c("library", "sex", "condition")

In [ ]:
gg.ls = list()

# Library
    i = 1
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(atac_obj$seurat_clusters, atac_obj$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

# Sex
    i = 2
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(atac_obj$seurat_clusters, atac_obj$sex))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

# Condition
    i = 3
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(atac_obj$seurat_clusters, atac_obj$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

In [ ]:
options(repr.plot.height = 20, repr.plot.width = 40)
cp <- gg.ls[[1]] + gg.ls[[2]] + gg.ls[[3]]
cp

In [ ]:
cell.sample = as.data.frame(table(atac_obj$library))
colnames(cell.sample)[1] = "sample.ID"

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)

width = 0.8
position = position_dodge(width = width)
gg1 = ggplot(cell.sample, aes(x= sample.ID, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Final Cells: ", length(colnames(atac_obj[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            ylim(0, 20000)+
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))+
            geom_text(hjust = -0.3, size = 2.9, position = position_dodge(width = width))+
            coord_flip()
gg1

In [ ]:
gc(reset = TRUE)

# Subcluster cluster 16
atac_obj_16_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=16,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')
# Subset the tmp obj 4 visualization
atac_obj_16_subcluster <- subset(atac_obj_16_subcluster, subset=seurat_clusters == 16)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_16_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_16_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
     


In [ ]:
# Install and load the cowplot package
install.packages("cowplot")
library(cowplot)

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_16_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
    p1 <- VlnPlot(atac_obj_16_subcluster, features='frac_reads_in_peaks', group.by='sub.cluster', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(atac_obj_16_subcluster$frac_reads_in_peaks), linetype='dashed', lw=2) + 
      theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
    p2 <- VlnPlot(atac_obj_16_subcluster, features='frac_reads_in_promoters', group.by='sub.cluster', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(atac_obj_16_subcluster$frac_reads_in_promoters), linetype='dashed', lw=2) + 
      theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
    p3 <- VlnPlot(atac_obj_16_subcluster, features='TSS.enrichment', group.by='sub.cluster', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(atac_obj_16_subcluster$TSS.enrichment), linetype='dashed', lw=2) + 
      theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
    p4 <- VlnPlot(atac_obj_16_subcluster, features='nucleosome_signal', group.by='sub.cluster', pt.size=0, log=TRUE) + geom_boxplot(width=.6, fill='white', alpha=.6, pt.size=0) + geom_hline(yintercept=median(atac_obj_16_subcluster$nucleosome_signal), linetype='dashed', lw=2) + 
      theme(plot.title = element_text(size = 25), axis.text = element_text(size=20))
    p5 = gg0
   combined_plot <- plot_grid(gg1, p5, ncol = 2)

   combined_plot 

In [ ]:
g = DotPlot(atac_obj_16_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")    

In [ ]:
# Library
    Covariant.table = as.data.frame(table(atac_obj_16_subcluster$sub.cluster, atac_obj_16_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

In [ ]:
# Condition
    Covariant.table = as.data.frame(table(atac_obj_16_subcluster$sub.cluster, atac_obj_16_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

In [ ]:
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot 

In [ ]:
gc(reset = TRUE)

# Subcluster 28
atac_obj_28_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=28,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')
# Subset the tmp obj 4 visualization
atac_obj_28_subcluster <- subset(atac_obj_28_subcluster, subset=seurat_clusters == 28)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_28_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_28_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_28_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_28_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")    

In [ ]:
# Library
    Covariant.table = as.data.frame(table(atac_obj_28_subcluster$sub.cluster, atac_obj_28_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

In [ ]:
# Condition
    Covariant.table = as.data.frame(table(atac_obj_28_subcluster$sub.cluster, atac_obj_28_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))

In [ ]:
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
gc(reset = TRUE)

# Subcluster immune
atac_obj_27_1_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=27,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')
# Subset the tmp obj 4 visualization
atac_obj_27_1_subcluster <- subset(atac_obj_27_1_subcluster, subset=seurat_clusters == 27)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_27_1_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_27_1_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_27_1_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .8)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_27_1_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_27_1_subcluster$sub.cluster, atac_obj_27_1_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_27_1_subcluster$sub.cluster, atac_obj_27_1_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
gc(reset = TRUE)

# Subcluster
atac_obj_29_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=29,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')
# Subset the tmp obj 4 visualization
atac_obj_29_subcluster <- subset(atac_obj_29_subcluster, subset=seurat_clusters == 29)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_29_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_29_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_29_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_29_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_29_subcluster$sub.cluster, atac_obj_29_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_29_subcluster$sub.cluster, atac_obj_27_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
names(atac_obj@graphs)

In [ ]:
# Load markers list
project.dir = "/nfs/lab/parulk/HPAP_scATAC/"
cell.markers = read.table("/nfs/lab/parulk/HPAP_scATAC/Immune.markers.txt", sep = ',', header = TRUE)
# Make it long, remove useless column and void markers
cell.markers <- cell.markers %>% gather(Key, marker, c(3:ncol(cell.markers)))
cell.markers = cell.markers[,-3]
cell.markers = cell.markers[cell.markers$marker != "", ]
head(cell.markers)
# Factorize columns
cell.markersCompartment <- cell.markers$cell.markersCompartment
cell.markersCellType <- cell.markers$cell.markersCellType
cell.markersCompartment = factor(cell.markersCompartment, levels = c("Immune"))
cell.markersCellType = factor(cell.markersCellType, levels = c("Megakaryocytes","Bcells","CD8","CD4","Tcells","Monocytes"))

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_27_1_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_27_1_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_27_1_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_27_1_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")


In [ ]:
options(repr.plot.height = 8, repr.plot.width = 25)

gg2

In [ ]:
g = DotPlot(atac_obj_27_1_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_27_1_subcluster$sub.cluster, atac_obj_27_1_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_27_subcluster$sub.cluster, atac_obj_27_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
table(Idents(atac_obj_27_1_subcluster))

In [ ]:
gc(reset = TRUE)

# Subcluster
atac_obj_26_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=26,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')

# Subset the tmp obj 4 visualization
atac_obj_26_subcluster <- subset(atac_obj_26_subcluster, subset=seurat_clusters == 26)
    

In [ ]:
g = DotPlot(atac_obj_26_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_26_subcluster$sub.cluster, atac_obj_26_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_26_subcluster$sub.cluster, atac_obj_26_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_26_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_26_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_26_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
gc(reset = TRUE)

# Subcluster
atac_obj_28_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=28,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')

# Subset the tmp obj 4 visualization
atac_obj_28_subcluster <- subset(atac_obj_28_subcluster, subset=seurat_clusters == 28)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_28_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_28_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_28_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_28_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_28_subcluster$sub.cluster, atac_obj_28_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_28_subcluster$sub.cluster, atac_obj_28_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
gc(reset = TRUE)

# Subcluster Endo
atac_obj_20_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=20,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')

# Subset the tmp obj 4 visualization
#atac_obj_20_subcluster <- subset(atac_obj_20_subcluster, subset=seurat_clusters == 20)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_20_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_20_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_20_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_20_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_20_subcluster$sub.cluster, atac_obj_20_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_20_subcluster$sub.cluster, atac_obj_20_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
gc(reset = TRUE)

# Subcluster Endo
atac_obj_18_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=18,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')

# Subset the tmp obj 4 visualization
#atac_obj_18_subcluster <- subset(atac_obj_18_subcluster, subset=seurat_clusters == 18)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_18_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_18_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_18_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_18_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_18_subcluster$sub.cluster, atac_obj_18_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_18_subcluster$sub.cluster, atac_obj_18_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
gc(reset = TRUE)

# Subcluster Endo
atac_obj_25_subcluster <- FindSubCluster(atac_obj, 
                                 cluster=25,
                                 resolution = 0.25,
                                 algorithm = 4,
                                 graph.name = 'ATAC_peaks_snn')

# Subset the tmp obj 4 visualization
#atac_obj_25_subcluster <- subset(atac_obj_25_subcluster, subset=seurat_clusters == 25)
    

In [ ]:
cell.cluster = as.data.frame(table(atac_obj_25_subcluster$sub.cluster))
colnames(cell.cluster)[1] = "sub.cluster"
cell.cluster$sub.cluster = factor(cell.cluster$sub.cluster)

width = 0.8
position = position_dodge(width = width)
gg0 = ggplot(cell.cluster, aes(x= sub.cluster, y = Freq, label = Freq)) + 
            theme_bw () +
            labs(y= "", x = "", title = paste("Total Cells: ", length(colnames(atac_obj_25_subcluster[["RNA"]])))) +
            geom_bar(stat="identity", 
                     position = position, width = width,
                     colour="black") +
            theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                  axis.text.x = element_text(angle = 90),
                             plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
options(repr.plot.height = 10, repr.plot.width = 14)
gg1 = DimPlot(atac_obj_25_subcluster,  group.by='sub.cluster', 
              label = TRUE, pt.size = .2)
combined_plot <- plot_grid(gg1, gg0, ncol = 2)

   combined_plot

In [ ]:
g = DotPlot(atac_obj_25_subcluster, assay='RNA', group.by = "sub.cluster",
            features=cell.markers$marker, cluster.idents=TRUE, col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    gg2 <- ggplot(meta_summary, aes(x = marker, reorder(id, -id)))+
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "lightgray", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
    labs(y= "Cluster_ID \n", x = "", title = "Markers's plot") + 
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 10, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 10),
            axis.text.y = element_text(size = 12, color = "black"),
            axis.title = element_text(size = 14)) +
      facet_nested(~ Compartment + CellType, scales = "free")
# Library
    Covariant.table = as.data.frame(table(atac_obj_25_subcluster$sub.cluster, atac_obj_25_subcluster$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg3 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Library") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
# Condition
    Covariant.table = as.data.frame(table(atac_obj_25_subcluster$sub.cluster, atac_obj_25_subcluster$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg4 = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = "Condition") + 
      theme(axis.text = element_text(size = 12), axis.title = element_text(size = 12, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 18, face = "bold", , hjust = 0.5))
 combined_plot <- plot_grid(gg2, gg3, gg4, ncol = 1)

   combined_plot

In [ ]:
#Remove cells, see slide for notes
atac_obj_tmp <- subset(x = atac_obj, idents = c(16, 26, 28, 29, 30, 31, 32), invert = TRUE)

In [ ]:
atac_obj_tmp

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)

DimPlot(object = atac_obj_tmp, label = TRUE,pt.size = 0.7 ) + NoLegend()

In [ ]:
atac_obj_tmp <- FindSubCluster(atac_obj_tmp, "25", subcluster.name = "sub_25_cluster", resolution = 0.25, algorithm = 4, graph.name = 'ATAC_peaks_snn')
DimPlot(atac_obj_tmp, label = TRUE, group.by= "sub_25_cluster", repel = TRUE, label.size = 3)

In [ ]:
atac_obj_tmp[["old.ident"]] <- Idents(atac_obj_tmp)

In [ ]:
Idents(atac_obj_tmp) <- "sub_25_cluster"

In [ ]:
table(Idents(atac_obj_tmp))

In [ ]:
atac_obj_tmp <- subset(x = atac_obj_tmp, idents = c("25_2", "25_3"), invert = TRUE)

In [ ]:
table(Idents(atac_obj_tmp))

In [ ]:
atac_obj_tmp <- FindSubCluster(atac_obj_tmp, "18", subcluster.name = "sub_18_cluster", resolution = 0.25, algorithm = 4, graph.name = 'ATAC_peaks_snn')
DimPlot(atac_obj_tmp, label = TRUE, group.by= "sub_18_cluster", repel = TRUE, label.size = 3)

In [ ]:
Idents(atac_obj_tmp) <- "sub_18_cluster"


In [ ]:
atac_obj_tmp <- subset(x = atac_obj_tmp, idents = c("18_5"), invert = TRUE)

In [ ]:
table(Idents(atac_obj_tmp))

In [ ]:
atac_obj_tmp <- FindSubCluster(atac_obj_tmp, "20", subcluster.name = "sub_20_cluster", resolution = 0.25, algorithm = 4, graph.name = 'ATAC_peaks_snn')
DimPlot(atac_obj_tmp, label = TRUE, group.by= "sub_20_cluster", repel = TRUE, label.size = 5)

In [ ]:
Idents(atac_obj_tmp) <- "sub_20_cluster"


In [ ]:
atac_obj_tmp

In [ ]:
atac_obj_tmp <- subset(x = atac_obj_tmp, idents = c("20_4"), invert = TRUE)

In [ ]:
atac_obj_tmp

In [ ]:
table(Idents(atac_obj_tmp))

In [ ]:
atac_obj_tmp[["old.ident.subcluster"]] <- Idents(atac_obj_tmp)

In [ ]:
Idents(atac_obj_tmp) <- "old.ident.subcluster"


In [ ]:
atac_obj_tmp

In [ ]:
DimPlot(atac_obj_tmp, label = TRUE,  repel = TRUE, label.size = 5)

In [ ]:
atac_obj_final <- RenameIdents(object = atac_obj_tmp,  '1' = 'Alpha', '2' = 'Alpha', '3' = 'Beta', '4' = 'Beta', '5' = 'Alpha', '6' = 'Acinar', '7' = 'Beta', '8' = 'Delta', '9' = 'Alpha', '10' = 'Ductal', '11' = 'Acinar', '12' = 'Ductal', '13' = 'Acinar', '14' = 'Alpha', '15' = 'Beta', '17' = 'Beta', '18_1'= 'A Stellate', '18_2'='A Stellate', '18_3'='A Stellate', '18_4' = 'Schwann', '19' = 'Ductal', '20_1' = 'Endothelial', '20_2' = 'Endothelial', '20_3' = 'Endothelial', '21' = 'MUC5B Ductal', '22' = 'Gamma', '23' = 'Alpha', '24' = 'Beta', '25_1' = 'Q Stellate', '27' = 'Immune')


In [ ]:
library(tidyverse)
library(patchwork)
install.packages("scCustomize")
install.packages("qs")
library(scCustomize)
library(qs)
BiocManager::install("dittoSeq")

In [ ]:
# Save as variable to global environment
options(repr.plot.width=20, repr.plot.height=15)
#varibow_pal <- DiscretePalette_scCustomize(num_colors = 35, palette = "varibow")
gg1_1 <- DimPlot_scCustom(seurat_object = atac_obj_final, shuffle = TRUE, label.size = 6.5, repel = T, label=FALSE,pt.size = 0.8, colors_use = DiscretePalette_scCustomize(num_colors = 50, palette = "varibow", shuffle=TRUE, seed = 88188188)) + theme(axis.text.x=element_text(angle=45, hjust=1), legend.key.size = unit(2,"line"), legend.text=element_text(size=20), plot.title = element_text(size = 30, face = "bold", hjust = 0.5)) + xlab('') + ylab('') 
g1 <- LabelClusters(gg1_1, id = "ident",  fontface = "bold", size = 9.5, repel = T, geom = "GeomPoint", position = "median")
gg1_1
g1

In [ ]:
covariant.ls = c("library", "sex", "condition")

In [ ]:
gg.ls = list()

# Library
    i = 1
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(atac_obj_final@active.ident, atac_obj_final$library))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 25), axis.title = element_text(size = 25, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 25, face = "bold", hjust = 0.5), legend.key.size = unit(3,"line"), legend.text=element_text(size=20))

# Sex
    i = 2
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(atac_obj_final@active.ident, atac_obj_final$sex))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 25), axis.title = element_text(size = 25, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 25, face = "bold", hjust = 0.5),legend.key.size = unit(2,"line"), legend.text=element_text(size=20))

# Condition
    i = 3
    covariant = covariant.ls[i]
    Covariant.table = as.data.frame(table(atac_obj_final@active.ident, atac_obj_final$condition))
    colnames(Covariant.table) = c("cluster", "covariant", "Freq")
    gg.ls[[i]] = ggplot(Covariant.table, aes(fill = covariant, y = Freq, x = cluster)) +
      theme_bw() +
      coord_flip() +
      geom_bar(position = position_fill(reverse = TRUE), stat = 'identity', color = 'black', size = 0.2) +
      labs(y= "\n Percentage", x = "", title = covariant) + 
      theme(axis.text = element_text(size = 25), axis.title = element_text(size = 25, face = "bold"),
                      axis.text.x = element_text(angle = 90),
                                 plot.title = element_text(size = 25, face = "bold",  hjust = 0.5), legend.key.size = unit(2,"line"), legend.text=element_text(size=20))

In [ ]:
options(repr.plot.height = 20, repr.plot.width = 40)
cp <- gg.ls[[1]] + gg.ls[[2]] + gg.ls[[3]]
cp

In [ ]:
out_dir <- "/nfs/lab/parulk/HPAP_scATAC/figures/"

ggsave(filename=paste0(out_dir, "dotplot.pdf"), plot = figure, device = "pdf", height= 15, width = 20, limitsize = FALSE)
ggsave(filename=paste0(out_dir, "dotplot.svg"), plot = figure, device = "svg", height= 15, width = 20, limitsize = FALSE)

In [ ]:
saveRDS(atac_obj_final, file='/nfs/lab/parulk/HPAP_scATAC/atac_obj_peak_based_final.rds')

In [ ]:
atac_obj <- readRDS('/nfs/lab/parulk/HPAP_scATAC/atac_obj_peak_based_final.rds')

In [ ]:
# Load markers list
project.dir = "/nfs/lab/parulk/HPAP_scATAC/"
cell.markers = read.table("/nfs/lab/parulk/HPAP_scATAC/Cell.markers.txt", sep = ',', header = TRUE)
# Make it long, remove useless column and void markers
cell.markers <- cell.markers %>% gather(Key, marker, c(3:ncol(cell.markers)))
cell.markers = cell.markers[,-3]
cell.markers = cell.markers[cell.markers$marker != "", ]
head(cell.markers)
# Factorize columns
cell.markersCompartment <- cell.markers$cell.markersCompartment
cell.markersCellType <- cell.markers$cell.markersCellType
cell.markersCompartment = factor(cell.markersCompartment, levels = c("Endocrine cells", "Non-endocrine cells"))
cell.markersCellType = factor(cell.markersCellType, levels = c("Beta", "Alpha", "Delta", "Gamma", "Ductal", "MUC5B Ductal", "Acinar",  "Act. Stellate", "Q. Stellate", "Endothelial", "Schwann", "Immune"))

In [ ]:
levels(atac_obj) <- c('Alpha', 'Beta', 'Delta', 'Gamma', 'Acinar', 'A Stellate', 'Ductal', 'Endothelial', 'Immune', 'MUC5B Ductal', 'Q Stellate', 'Schwann')

In [ ]:
g = DotPlot(atac_obj, assay='RNA', features=cell.markers$marker,  col.min=0) +
        theme(axis.text.x=element_text(angle=45, hjust=1)) + xlab('') + ylab('')
    meta_summary = g$data
    colnames(meta_summary)[3] = "marker"
    meta_summary = merge(meta_summary, cell.markers, by = "marker")

    options(repr.plot.width=25, repr.plot.height=10)
    figure <- ggplot(meta_summary, aes(x = marker, y = id)) +
      geom_point(aes(size = pct.exp, fill = avg.exp.scaled, stroke=NA),
                 shape = 21) +
      scale_size("% detected", range = c(0, 6)) +
      scale_fill_gradient(low = "white", high = "blue",
                           guide = guide_colorbar(nbin = 200,
                                                  ticks.colour = "black", frame.colour = "black"),
                           name = "Average\nexpression") +
      ylab("Cluster") + xlab("") +
      theme_bw() +
      theme(axis.text = element_text(size = 100),
            axis.text.x = element_text(size = 20, angle = 45, hjust = 1, color = "black"),
            strip.text.x = element_text(size = 13),
            axis.text.y = element_text(size = 20, color = "black"),
            axis.title = element_text(size = 20), legend.key.size = unit(3,"line"), legend.text=element_text(size=15)) +
facet_nested(~ Compartment + CellType, scales = "free")

figure

In [ ]:
table(Idents(atac_obj))

In [ ]:
#add AAB status
aab_positive = list('HPAP-045', 'HPAP-049', 'HPAP-050','HPAP-072', 'HPAP-092')
aab_negative = list('HPAP-035', 'HPAP-036', 'HPAP-039','HPAP-040', 'HPAP-044', 'HPAP-047', 'HPAP-051', 'HPAP-052', 'HPAP-053', 'HPAP-054', 'HPAP-055', 'HPAP-056', 'HPAP-059', 'HPAP-061', 'HPAP-062', 'HPAP-063', 'HPAP-064', 'HPAP-067', 'HPAP-069', 'HPAP-075', 'HPAP-077', 'HPAP-079', 'HPAP-080', 'HPAP-081', 'HPAP-083', 'HPAP-084', 'HPAP-085', 'HPAP-088', 'HPAP-099', 'HPAP-100', 'HPAP-101', 'HPAP-103', 'HPAP-104', 'HPAP-105', 'HPAP-106', 'HPAP-109'  )


atac_obj@meta.data$AAB[atac_obj@meta.data$library %in% aab_positive] <- 'Positive'
atac_obj@meta.data$AAB[atac_obj@meta.data$library %in% aab_negative] <- 'Negative'
head(atac_obj@meta.data)
head(atac_obj@meta.data$AAB)


In [ ]:
table((atac_obj$AAB))

In [ ]:
saveRDS(atac_obj, file='/nfs/lab/parulk/HPAP_scATAC/atac_obj_peak_based_metadata_final.rds')

In [ ]:
atac_obj <- readRDS('/nfs/lab/parulk/HPAP_scATAC/atac_obj_peak_based_metadata_final.rds')